In [121]:
import fitz
import re
# Đường dẫn đến file PDF của bạn
pdf_path = './Data/SKL007296.pdf'

In [122]:
def extract_pdf_text(pdf_path):
    pdf_document = fitz.open(pdf_path)
    extracted_text = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text().strip()
        if text:
            extracted_text.append((text, page_num + 1))  # page_num + 1 để đánh số trang từ 1
    pdf_document.close()
    return extracted_text
def save_text_with_page_to_file(lines_with_page, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for line, page_num in lines_with_page:
            file.write(f"Số trang {page_num}:\n{line}\n\n")

# Trích xuất nội dung văn bản từ file PDF với số trang
text_with_page = extract_pdf_text(pdf_path)

# Lưu nội dung vào file văn bản với số trang
save_text_with_page_to_file(text_with_page, './output/content.txt')

In [123]:
import re

def combine_lines_and_split_sentences(text_with_page):
    combined_lines = []
    current_sentence = ''

    for text, page_num in text_with_page:
        lines = text.split('\n')
        for line in lines:
            # Loại bỏ các dòng chỉ chứa dấu chấm câu và khoảng trắng
            if re.match(r'^[\s]*$', line):
                continue
            # Xóa khoảng trắng thừa với dấu câu và loại bỏ ký tự đặc biệt và số La Mã
            line = re.sub(r'\.{2,}|\s{2,}', ' ', line)
            line = re.sub(r'[^\w\s]|([ivxIVX]+\b)', '', line)           
            
            # Tách câu nếu dòng không bắt đầu bằng số và kết hợp các dòng liên tiếp
            if line.strip():  # Only process non-empty lines
                if line[0].isupper() or line[0] in '“"':
                    if current_sentence:
                        combined_lines.append((current_sentence.strip(), page_num))
                    current_sentence = line.strip()
                else:
                    current_sentence += ' ' + line.strip()
    
    if current_sentence:
        combined_lines.append((current_sentence.strip(), page_num))
    
    return combined_lines

def save_combined_text_with_page_to_file(combined_text_with_page, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for i, (sentence, page_num) in enumerate(combined_text_with_page, start=1):
            file.write(f"Trang {page_num}, Câu {i}:{sentence}\n")

# Kết hợp các dòng và tách câu, lưu cả số trang cho mỗi câu
sentences_with_page = combine_lines_and_split_sentences(text_with_page)

# Lưu nội dung vào file văn bản với số dòng và số trang
save_combined_text_with_page_to_file(sentences_with_page, './output/sentence_split.txt')


In [124]:
def remove_single_word_sentences(sentences):
    return [sentence for sentence in sentences if len(sentence[0].split()) > 1]

def add_line_numbers(sentences_with_page):
    numbered_sentences = []
    for i, (sentence, page_num) in enumerate(sentences_with_page, start=1):
        numbered_sentences.append(f"Trang {page_num}: Câu {i}: {sentence}")
    return numbered_sentences

def save_processed_sentences_to_file(sentences_with_page, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for sentence in sentences_with_page:
            file.write(f"{sentence}\n")
            
# Loại bỏ các câu có ít hơn 1 từ
processed_sentences = remove_single_word_sentences(sentences_with_page)

# Thêm số dòng và số trang vào mỗi câu
numbered_sentences = add_line_numbers(processed_sentences)

# Lưu nội dung vào file văn bản
save_processed_sentences_to_file(numbered_sentences, './output/processed_sentences.txt')


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Chuẩn bị dữ liệu văn bản
def tfidf_features(texts):
    vectorizer = TfidfVectorizer()
    features = vectorizer.fit_transform(texts)
    return features

# Sử dụng vectorizer để biến đổi văn bản mới
new_texts = ["l 0 0 2 1 5 4"]
new_features = tfidf_features(new_texts)

# In ra ma trận đặc trưng của văn bản mới
print(new_features.toarray())


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [11]:
import spacy

# Load Vietnamese spaCy model
nlp = spacy.blank("vi")

# Danh sách từ dừng tự định nghĩa
custom_stop_words = {"của", "và", "là", "các", "cho", "với", "làm", "để"}  # Thêm các từ dừng khác cần thiết

# In ra các từ dừng có sẵn trong vocab của spaCy
if nlp.vocab and nlp.vocab.stop_words:
    print("Danh sách từ dừng của mô hình spaCy:")
    print(nlp.vocab.stop_words)

# Hoặc in ra các từ dừng từ danh sách tự định nghĩa
print("Danh sách từ dừng tự định nghĩa:")
print(custom_stop_words)


Danh sách từ dừng tự định nghĩa:
{'các', 'để', 'cho', 'của', 'làm', 'và', 'là', 'với'}
